<a href="https://colab.research.google.com/github/tonybuii2003/AITourGuide/blob/main/tourguidegpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

In [ ]:
pip install openai


In [ ]:
from openai import AsyncOpenAI
import asyncio

In [ ]:
import os

In [ ]:
data = pd.read_csv('/content/MetObjects.csv')

<ipython-input-5-0aa2d8616974>:1: DtypeWarning: Columns (5,11,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/MetObjects.csv')


In [ ]:
print(data.columns.values)

['Object Number' 'Is Highlight' 'Is Timeline Work' 'Is Public Domain'
 'Object ID' 'Gallery Number' 'Department' 'AccessionYear' 'Object Name'
 'Title' 'Culture' 'Period' 'Dynasty' 'Reign' 'Portfolio' 'Constituent ID'
 'Artist Role' 'Artist Prefix' 'Artist Display Name' 'Artist Display Bio'
 'Artist Suffix' 'Artist Alpha Sort' 'Artist Nationality'
 'Artist Begin Date' 'Artist End Date' 'Artist Gender' 'Artist ULAN URL'
 'Artist Wikidata URL' 'Object Date' 'Object Begin Date' 'Object End Date'
 'Medium' 'Dimensions' 'Credit Line' 'Geography Type' 'City' 'State'
 'County' 'Country' 'Region' 'Subregion' 'Locale' 'Locus' 'Excavation'
 'River' 'Classification' 'Rights and Reproduction' 'Link Resource'
 'Object Wikidata URL' 'Metadata Date' 'Repository' 'Tags' 'Tags AAT URL'
 'Tags Wikidata URL']


In [ ]:
relevant_columns = [
    'Title', 'Culture', 'Period', 'Dynasty', 'Reign',
    'Artist Display Name', 'Artist Nationality', 'Artist Display Bio',
    'Object Date', 'Medium', 'Dimensions', 'Credit Line',
    'City', 'Country', 'Region', 'Classification', 'Tags'
]

# Fill NaN values with empty strings to avoid errors during concatenation
data[relevant_columns] = data[relevant_columns].fillna('')

# Create a combined_text column with prioritized information
data['combined_text'] = data.apply(
    lambda row: ' | '.join([f"{col}: {row[col]}" for col in relevant_columns if row[col]]),
    axis=1
)

In [ ]:
# Step 1: Retriever - TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['combined_text'])

In [ ]:
pip install httpx

In [ ]:
import httpx

In [ ]:
client = AsyncOpenAI(api_key=API_KEY)

In [ ]:
async def rag_query(user_query):
    query_vector = vectorizer.transform([user_query])

    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()

    top_doc_index = similarity_scores.argmax()
    retrieved_context = data.iloc[top_doc_index]['combined_text']
    prompt = f"Context: {retrieved_context}\n\nQuestion: {user_query}\nAnswer:"
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert tour guide at The Metropolitan Museum of Art (The Met) in New York City. Your role is to provide engaging, accurate, and detailed information about the museum's exhibits, history, and artworks. Answer questions with enthusiasm, clarity, and a friendly tone, making the information accessible to all ages. Highlight interesting facts, cultural significance, and historical context when relevant. Feel free to recommend must-see exhibits, hidden gems, and tips for enjoying the museum experience. If the visitor asks in a specific language, respond in the same language fluently while maintaining the same level of detail and professionalism. Always adapt your tone to be culturally appropriate and respectful. If you don’t have enough information, respond honestly and encourage the visitor to explore further at The Met."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content



In [ ]:
# Example Query
query = "kể tôi về 3 bức tranh có giá trị lịch sử lâu đời nhất ở Met"

try:
    import nest_asyncio
    nest_asyncio.apply()
    response = await rag_query(query)
except RuntimeError:
    response = asyncio.run(rag_query(query))

print(response)

Cảm ơn bạn đã quan tâm đến những tác phẩm nghệ thuật có giá trị lịch sử tại The Met! Tuy nhiên, cần lưu ý rằng việc xác định ba bức tranh có giá trị lịch sử lâu đời nhất có thể khá khó khăn, bởi vì The Met sở hữu một bộ sưu tập rất rộng lớn với nhiều tác phẩm từ khắp nơi trên thế giới và qua nhiều thời kỳ lịch sử khác nhau. Dưới đây là ba tác phẩm nổi bật với giá trị lịch sử cao mà bạn có thể quan tâm:

1. **"Madonna and Child" của Duccio di Buoninsegna** - Đây là một tác phẩm quan trọng của nghệ sĩ người Ý thời kỳ đầu. Bức tranh này thể hiện sự chuyển mình từ nghệ thuật Byzantine sang phong cách hội họa Phục Hưng sớm, và là ví dụ điển hình cho tài năng của Duccio trong việc sử dụng màu sắc và biểu cảm.

2. **"Aristotle with a Bust of Homer" của Rembrandt** - Được thực hiện vào năm 1653, đây là một trong những tác phẩm quan trọng của Rembrandt, nghệ sĩ người Hà Lan. Bức tranh này được biết đến với kỹ thuật ánh sáng và sự diễn tả sâu sắc về cảm xúc và tư tưởng.

3. **"The Death of Socra